In [1]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import pickle as pkl
import random

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.neighbors import NearestNeighbors

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("anime_ml.csv")
print(df.shape)
df.head()

(11903, 49)


,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,Music.1,ONA,OVA,Special,TV
0,32281,Kimi no Na wa.,1,9.37,200630,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,64,9.26,793665,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,28977,Gintama,51,9.25,114262,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,9253,Steins Gate,24,9.17,673572,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,9969,Gintama',51,9.16,151266,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [3]:
df.describe()

,anime_id,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,Music.1,ONA,OVA,Special,TV
count,11903.000000,11903.000000,11903.000000,1.190300e+04,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000
mean,13515.515080,12.426279,6.484570,1.844488e+04,0.233555,0.193901,0.005965,0.378728,0.019995,0.024364,0.166765,0.189028,0.014618,0.066454,0.030497,0.004369,0.130975,0.063849,0.021675,0.078216,0.035033,0.070234,0.040830,0.033437,0.016382,0.019071,0.121314,0.012182,0.098715,0.171049,0.044695,0.049567,0.004537,0.141057,0.005377,0.099555,0.031589,0.044611,0.037554,0.084601,0.007225,0.008485,0.193397,0.040914,0.051164,0.273881,0.140217,0.300428
std,11174.446066,46.959892,1.016017,5.521622e+04,0.423110,0.395369,0.077005,0.485091,0.139989,0.154182,0.372781,0.391547,0.120024,0.249084,0.171956,0.065954,0.337388,0.244495,0.145627,0.268522,0.183871,0.255552,0.197905,0.179782,0.126946,0.136780,0.326505,0.109701,0.298291,0.376568,0.206641,0.217058,0.067205,0.348095,0.073132,0.299418,0.174910,0.206456,0.190122,0.278298,0.084696,0.091728,0.394978,0.198099,0.220341,0.445967,0.347226,0.458464
min,1.000000,1.000000,1.670000,5.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3361.500000,1.000000,5.900000,2.340000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9885.000000,2.000000,6.570000,1.596000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,23485.000000,12.000000,7.180000,9.830000e+03,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,34527.000000,1818.000000,10.000000,1.013917e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000

In [4]:
def make_recommendation(anime_length, anime_name=None, genres=None, type_filter=None, rating_min=None, episodes_min=None):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + (genres if genres else [])  # Include genres if provided

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Determine which anime to base recommendations on
    if anime_name:
        anime_match = df.loc[df.name == anime_name]
        if anime_match.empty:
            raise ValueError(f"Anime '{anime_name}' not found.")
        anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]
    else:
        # Randomly select an anime if no name is provided
        anime_id = random.choice(df['anime_id'].values)

    # Apply genre filter if specified
    filtered_df = df
    if genres is not None and genres:
        genre_filter = filtered_df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
        filtered_df = filtered_df[genre_filter]

    # Apply type filter if specified
    if type_filter is not None and type_filter:
        type_conditions = [filtered_df[type] == True for type in type_filter if type in filtered_df.columns]
        if type_conditions:
            combined_condition = np.logical_or.reduce(type_conditions)
            filtered_df = filtered_df[combined_condition]

    # Apply minimum rating filter if specified
    if rating_min is not None:
        filtered_df = filtered_df[filtered_df['rating'] >= rating_min]

    # Apply minimum episodes filter if specified
    if episodes_min is not None:
        filtered_df = filtered_df[filtered_df['episodes'] >= episodes_min]

    # If no anime matches the filters, raise an error
    if filtered_df.empty:
        raise ValueError("No anime found for the specified filters.")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

In [5]:
# Example usage
anime_length = 10
anime_name = None  # Can be None if no specific anime is needed
genres = ["Comedy"]  # Specify the genres you want to filter by (can be None)
type_filter = None  # Specify the types to filter by (can be None)
rating_min = None  # Minimum rating (can be None if no minimum is needed)
episodes_min = 1  # Minimum episodes (can be None if no minimum is needed)

response = make_recommendation(anime_length, anime_name=anime_name, genres=genres, type_filter=type_filter, rating_min=rating_min, episodes_min=episodes_min)

In [6]:
# To test
pd.DataFrame(response)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,ONA,OVA,Special,TV,distance
0,10534,Doraemon Movie 31: Shin Nobita to Tetsujin Hei...,1,7.73,1317,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0.082237
1,31997,Hetalia: The World Twinkle Extra Disc,1,7.71,2152,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.096596
2,1924,Urusei Yatsura Movie 5: Final,1,7.72,2783,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.097237
3,7303,Detective Conan: Black History,1,7.74,3603,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.131757
4,33454,Kochira Katsushikaku Kameari Kouenmae Hashutsu...,1,7.77,268,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0.132620
5,25341,Diamond no Ace OVA,3,7.72,7376,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0.142330
6,2313,Muumindani no Suisei,1,7.74,4344,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.144500
7,9786,Detective Conan: Conan vs. Kid - Jet Black Sniper,1,7.66,2119,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0.147078
8,2673,Doraemon Movie 27: Nobita no Shin Makai Daibou...,1,7.65,1583,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.148070
9,1478,City Hunter: Goodbye My Sweetheart,1,7.67,2832,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.149129


In [7]:
with open('anime.pkl', 'wb') as file:
        pkl.dump(df, file)